# Volume 1, Chapter 10: API Integration Patterns

**Connect AI to Netmiko, NAPALM, and REST APIs**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/master/Volume-1-Foundations/Colab-Notebooks/Vol1_Ch10_API_Integration.ipynb)

---

**What you'll learn:**
- 🔌 Integrate AI with Netmiko
- 🔄 AI-powered config validation
- 🌐 Build REST APIs for AI services
- 🤖 Intelligent troubleshooting workflows

In [ ]:
!pip install -q anthropic

import os
from getpass import getpass

try:
    from google.colab import userdata
    os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
except:
    if 'ANTHROPIC_API_KEY' not in os.environ:
        os.environ['ANTHROPIC_API_KEY'] = getpass('Anthropic API key: ')

from anthropic import Anthropic
client = Anthropic()
print("✓ Ready!")

---
## 🔌 Example 1: AI + Netmiko Pattern (Simulated)

In [ ]:
# Simulated Netmiko output (in real usage, this comes from device)
def simulate_netmiko(command):
    """Simulate Netmiko command output."""
    outputs = {
        "show ip bgp summary": """BGP router identifier 10.1.1.1, local AS number 65001
Neighbor        V    AS MsgRcvd MsgSent   TblVer  InQ OutQ Up/Down  State/PfxRcd
10.1.1.2        4 65002    1500    1495      100    0    0 00:05:30 Idle
10.1.1.3        4 65003    2000    1998      100    0    0 2d15h    150""",
        
        "show ip route 10.1.1.2": """Routing entry for 10.1.1.0/30
  Known via "connected", distance 0, metric 0
  Directly connected via GigabitEthernet0/0""",
        
        "show logging | include BGP": """Jan 15 10:20:00: %BGP-5-ADJCHANGE: neighbor 10.1.1.2 Down BGP Notification sent
Jan 15 10:20:00: %BGP-3-NOTIFICATION: sent to neighbor 10.1.1.2 6/4 (Administrative Reset)"""
    }
    return outputs.get(command, "Command not found")

class AINetworkAssistant:
    def __init__(self):
        self.command_history = []
    
    def diagnose(self, symptom):
        """AI-driven troubleshooting."""
        
        # Step 1: Ask AI what commands to run
        response = client.messages.create(
            model="claude-haiku-4-5-20251001",
            max_tokens=200,
            temperature=0,
            messages=[{
                "role": "user",
                "content": f"""Network symptom: {symptom}

What 3 Cisco IOS commands should I run to diagnose this?
Return ONLY the commands, one per line, no explanation."""
            }]
        )
        
        commands = [cmd.strip() for cmd in response.content[0].text.strip().split('\n') if cmd.strip()]
        
        # Step 2: Run commands (simulated)
        outputs = {}
        for cmd in commands[:3]:
            outputs[cmd] = simulate_netmiko(cmd)
            self.command_history.append(cmd)
        
        # Step 3: AI analyzes results
        analysis_prompt = f"""Symptom: {symptom}

Command outputs:
"""
        for cmd, output in outputs.items():
            analysis_prompt += f"\n### {cmd}\n{output}\n"
        
        analysis_prompt += "\nAnalyze these outputs. What's the root cause and fix?"
        
        analysis = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=500,
            temperature=0,
            messages=[{"role": "user", "content": analysis_prompt}]
        )
        
        return {
            "commands_run": commands[:3],
            "outputs": outputs,
            "analysis": analysis.content[0].text
        }

# Test
assistant = AINetworkAssistant()
result = assistant.diagnose("BGP neighbor 10.1.1.2 is showing Idle state")

print("🔌 AI + NETMIKO INTEGRATION")
print("=" * 60)
print(f"\nCommands AI suggested: {result['commands_run']}")
print(f"\n📊 Analysis:")
print(result['analysis'])

---
## 🔄 Example 2: Config Validation Before Apply

In [ ]:
def validate_config_change(current_config, proposed_change):
    """Validate config change before applying."""
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=500,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Review this config change for safety:

CURRENT CONFIG:
{current_config}

PROPOSED CHANGE:
{proposed_change}

Check for:
1. Syntax errors
2. Security risks
3. Potential outage risks
4. Best practice violations

Return JSON:
{{"safe": true/false, "issues": ["list of issues"], "recommendation": "approve/reject/modify"}}"""
        }]
    )
    
    import json
    import re
    text = response.content[0].text
    json_match = re.search(r'\{.*\}', text, re.DOTALL)
    return json.loads(json_match.group()) if json_match else {"safe": False, "issues": ["Parse error"]}

# Test
current = """
interface GigabitEthernet0/0
 ip address 192.168.1.1 255.255.255.0
 no shutdown
"""

proposed = """
interface GigabitEthernet0/0
 ip address 10.0.0.1 255.255.255.0
 shutdown
"""

result = validate_config_change(current, proposed)

print("🔄 CONFIG VALIDATION")
print("=" * 60)
print(f"Safe: {result.get('safe')}")
print(f"Recommendation: {result.get('recommendation')}")
print(f"Issues:")
for issue in result.get('issues', []):
    print(f"  ⚠️ {issue}")

---
## 🌐 Example 3: REST API Pattern (FastAPI)

In [ ]:
# This shows the pattern - in production, run with uvicorn

api_code = '''
from fastapi import FastAPI
from pydantic import BaseModel
from anthropic import Anthropic

app = FastAPI()
client = Anthropic()

class ConfigRequest(BaseModel):
    config: str
    check_type: str = "security"  # security, compliance, best_practice

class AnalysisResponse(BaseModel):
    issues: list
    risk_level: str
    recommendations: list

@app.post("/analyze", response_model=AnalysisResponse)
def analyze_config(request: ConfigRequest):
    response = client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=500,
        messages=[{
            "role": "user",
            "content": f"Analyze this config for {request.check_type} issues:\\n{request.config}"
        }]
    )
    # Parse response and return structured data
    return {"issues": [], "risk_level": "low", "recommendations": []}

# Run with: uvicorn api:app --port 8000
'''

print("🌐 FASTAPI PATTERN")
print("=" * 60)
print(api_code)
print("\n📌 Save as api.py and run: uvicorn api:app --port 8000")

---
## 🤖 Example 4: Intelligent Change Workflow

In [ ]:
def change_workflow(intent, current_config):
    """AI-assisted change management."""
    
    # Step 1: Generate config from intent
    gen_response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=500,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Generate Cisco IOS config for this intent:
{intent}

Current config context:
{current_config}

Return ONLY the config commands to add/change."""
        }]
    )
    proposed = gen_response.content[0].text
    
    # Step 2: Validate
    val_response = client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=300,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Is this config safe to apply? Any syntax errors?
{proposed}

Return: SAFE or UNSAFE with reason."""
        }]
    )
    validation = val_response.content[0].text
    
    # Step 3: Generate rollback
    rollback_response = client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=300,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Generate rollback commands for this change:
{proposed}

Return ONLY the rollback commands."""
        }]
    )
    rollback = rollback_response.content[0].text
    
    return {
        "intent": intent,
        "proposed_config": proposed,
        "validation": validation,
        "rollback_commands": rollback
    }

# Test
result = change_workflow(
    intent="Add VLAN 100 named ENGINEERING with IP 10.100.1.1/24",
    current_config="hostname SWITCH-01\nvlan 10\n name MANAGEMENT"
)

print("🤖 INTELLIGENT CHANGE WORKFLOW")
print("=" * 60)
print(f"\nIntent: {result['intent']}")
print(f"\n📋 Proposed Config:")
print(result['proposed_config'])
print(f"\n✅ Validation: {result['validation'][:100]}...")
print(f"\n↩️ Rollback Commands:")
print(result['rollback_commands'])

---
## 🎯 Key Takeaways

| Pattern | Use Case | Benefit |
|---------|----------|--------|
| AI + Netmiko | Troubleshooting | Dynamic command selection |
| Config Validation | Change management | Catch errors before apply |
| REST API | Team access | Centralized AI service |
| Change Workflow | Automation | Intent → Config → Validate → Rollback |

**Safety first:**
- Always validate AI-generated configs
- Require human approval for production changes
- Generate rollback before applying

---

## 📚 Next Steps

➡️ [Chapter 11: Testing and Validation](./Vol1_Ch11_Testing.ipynb)